# Gene Regulatory Network Analysis: Knockout Effects

This notebook demonstrates the analysis of gene knockout effects using GenKI framework.

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.genki_inference import GenKIAnalyzer
from src.visualization import NetworkVisualizer

## 1. Load Data and GRN

In [ ]:
# Set tissue
tissue = "root"  # Change this for different tissues

# Load expression data
adata_path = f"data/processed/{tissue}_processed.h5ad"

# Load GRN
grn_path = f"results/grn/{tissue}/grn_edges.csv"
grn_df = pd.read_csv(grn_path)

# Load target genes
target_genes = pd.read_csv("data/target_genes.csv")['GeneID'].tolist()

print(f"Number of target genes: {len(target_genes)}")

## 2. Initialize GenKI Analysis

In [ ]:
# Initialize GenKI analyzer
analyzer = GenKIAnalyzer(
    epochs=600,
    learning_rate=7e-4,
    beta=1e-4,
    seed=8096
)

# Setup output directories
output_dir = f"results/genki/{tissue}"
model_dir = f"models/genki/{tissue}"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

## 3. Run GenKI Analysis

In [ ]:
# Run analysis for target genes
genki_results = analyzer.process_multiple_targets(
    adata_path=adata_path,
    grn_path=grn_path,
    target_genes=target_genes,
    output_dir=output_dir,
    model_dir=model_dir
)

print("Analysis completed for all target genes")

## 4. Validate GRN Edges

In [ ]:
# Validate edges using GenKI results
validated_grn = analyzer.validate_grn_edges(grn_df, output_dir)

# Display validation statistics
total_edges = len(validated_grn)
validated_edges = (validated_grn['GenKIValid'] == 1).sum()
print(f"Total edges: {total_edges}")
print(f"Validated edges: {validated_edges} ({validated_edges/total_edges*100:.1f}%)")

## 5. Analyze Validation Results

In [ ]:
# Initialize visualizer
visualizer = NetworkVisualizer()

# Plot validation summary
plt.figure(figsize=(10, 10))
visualizer.plot_genki_validation_summary(validated_grn)
plt.title(f"GenKI Validation Summary - {tissue.capitalize()} Tissue")
plt.show()

# Plot distribution of PearsonR by validation status
plt.figure(figsize=(10, 6))
visualizer.plot_genki_distribution(validated_grn)
plt.title(f"Expression Correlation Distribution by Validation Status - {tissue.capitalize()} Tissue")
plt.show()

## 6. Analyze Individual Target Genes

In [ ]:
# Select example target gene
example_target = target_genes[0]

# Load specific results
target_results = pd.read_csv(f"{output_dir}/{example_target}_res.csv")

# Plot network for target gene
plt.figure(figsize=(15, 15))
visualizer.plot_network(
    validated_grn,
    target_gene=example_target,
    min_appearance=0.8
)
plt.title(f"Validated Network for {example_target}")
plt.show()

## 7. Cross-Tissue Comparison

In [ ]:
# Load validation results from different tissues
tissues = ['root', 'leaf', 'seed', 'shoot']
tissue_results = {}

for t in tissues:
    results_path = f"results/genki/{t}/gene_importance_genki.csv"
    if os.path.exists(results_path):
        tissue_results[t] = pd.read_csv(results_path)

# Compare validation results across tissues
if len(tissue_results) > 1:
    plt.figure(figsize=(12, 8))
    visualizer.plot_tissue_comparison(tissue_results)
    plt.title("Validated Negative Regulators Across Tissues")
    plt.show()

## 8. Save Final Results

In [ ]:
# Save validated network
validated_grn.to_csv(f"{output_dir}/validated_network.csv", index=False)

# Create summary statistics
summary_stats = {
    'tissue': tissue,
    'total_edges': total_edges,
    'validated_edges': validated_edges,
    'validation_rate': validated_edges/total_edges,
    'negative_regulators': (validated_grn['Regulator'] == 'negative').sum(),
    'validated_negative': ((validated_grn['Regulator'] == 'negative') & 
                          (validated_grn['GenKIValid'] == 1)).sum()
}

# Save summary statistics
pd.DataFrame([summary_stats]).to_csv(f"{output_dir}/validation_summary.csv", index=False)

print(f"Results saved to {output_dir}")